# 1️⃣ Project Setup and Data Loading

***

### 📝 **Goal:** Load the cleaned data. We ensure that **Pandas** is correctly imported and that the file paths lead to the final, clean datasets from the Data Preparation phase.

In [30]:
## 1. Setup and Data Loading

import pandas as pd
import numpy as np

# Define paths to the final cleaned files
OPERATING_EXPENSES_PATH = "../1_datasets/processed/operating-expenses_cleaned_final.csv"
CAPITAL_PATH = "../1_datasets/processed/capital_cleaned_final_V2.csv"

# Read the files
try:
    df_exp = pd.read_csv(OPERATING_EXPENSES_PATH)
    df_cap = pd.read_csv(CAPITAL_PATH)
    print("Data successfully loaded. Ready for exploration.")
except FileNotFoundError as e:
    print(f"Error loading file: {e}. Please check the paths.")

Data successfully loaded. Ready for exploration.


# 2️⃣ Data Structure Review

***

### 🔍 **Goal:** Perform a final quality check using `df.info()`. This step verifies:
1.  **Data Types (Dtypes):** All columns are in the expected format (e.g., prices are `float64`).
2.  **Non-Null Counts:** Confirmation that missing value imputation was successful in the required columns.

In [31]:
## 2. Data Structure Review

print("--- 2.1. Operating Expenses (df_exp) Info ---")
df_exp.info()

print("\n--- 2.2. Capital Assets (df_cap) Info ---")
df_cap.info()

--- 2.1. Operating Expenses (df_exp) Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Arabic Item     46 non-null     object 
 1   English Item    46 non-null     object 
 2   Category        46 non-null     object 
 3   Date            34 non-null     object 
 4   Quantity        13 non-null     float64
 5   Unit            8 non-null      object 
 6   Price / Amount  46 non-null     float64
 7   Notes           2 non-null      object 
dtypes: float64(2), object(6)
memory usage: 3.0+ KB

--- 2.2. Capital Assets (df_cap) Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Item Arabic   145 non-null    int64  
 1   Item English  31 non-null     object 
 2   Category      145 non-null

# 3️⃣ Operating Expenses Deep Dive: **Cost Focus**

***

### 💰 **Goal:** Summarize operational spending. We identify the average expense and, crucially, determine the **Top 5 Costliest Categories**. This "Danger Zone" analysis highlights areas for immediate cost control efforts.

In [32]:
## 3. Operating Expenses: Financial Overview

# Descriptive Statistics for Price / Amount
print("--- 3.1. Descriptive Statistics for Price / Amount ---")
print(df_exp['Price / Amount'].describe())

# Top 5 Costliest Categories by Total Sum
print("\n--- 3.2. Top 5 Costliest Expense Categories ---")
top_categories = df_exp.groupby('Category')['Price / Amount'].sum().sort_values(ascending=False).head(5)
print(top_categories.to_string())

# Total Expenditure
total_expenditure = df_exp['Price / Amount'].sum()
print(f"\nTOTAL OPERATING EXPENDITURE: ${total_expenditure:,.2f}")

--- 3.1. Descriptive Statistics for Price / Amount ---
count    4.600000e+01
mean     1.348990e+05
std      2.715244e+05
min      2.000000e+00
25%      5.725000e+01
50%      6.675000e+02
75%      2.000000e+05
max      1.576000e+06
Name: Price / Amount, dtype: float64

--- 3.2. Top 5 Costliest Expense Categories ---
Category
Rent             1930000.0
Name/Person      1675000.0
Equipment        1585500.0
Material          824132.0
Miscellaneous     190000.0

TOTAL OPERATING EXPENDITURE: $6,205,352.00


# 4️⃣ Monthly Expense Trend: **Time Analysis**

***

### 📅 **Goal:** Analyze spending over time. By aggregating expenses monthly, we can detect any sudden spikes, seasonal changes, or unexpected **Outliers**. This forms the basis for accurate financial forecasting.

In [33]:
## 4. Monthly Expense Trend

# Ensure Date column is in datetime format and extract the Month
df_exp['Date'] = pd.to_datetime(df_exp['Date'], errors='coerce')
df_exp['Month'] = df_exp['Date'].dt.month

print("--- 4.1. Total Spending per Month ---")

# Group spending by month
monthly_spending = df_exp.groupby('Month')['Price / Amount'].sum().sort_index()

# Display monthly changes
print(monthly_spending.to_string())

# Identify maximum spending month
max_month = monthly_spending.idxmax()
max_amount = monthly_spending.max()
print(f"\nMonth with MAX Spending: Month {max_month} (${max_amount:,.2f})")

--- 4.1. Total Spending per Month ---
Month
6.0    2451000.0
7.0    3040394.0
8.0     709734.0
9.0         37.0

Month with MAX Spending: Month 7.0 ($3,040,394.00)


# 5️⃣ Capital Assets Overview: **Investment Focus**

***

### ⚙️ **Goal:** Summarize the **Capital Investment** profile of the 'HajarAsas' factory. We check the total investment value, verify the largest single asset purchase, and identify the top categories where capital was allocated (e.g., Machinery, Construction).

In [34]:
## 5. Capital Assets: Investment Overview

# Descriptive Statistics for Price (Capital Assets)
print("--- 5.1. Descriptive Statistics for Price (Capital) ---")
print(df_cap['Price'].describe())

# Total Value of all Priced Assets
total_capital_investment = df_cap['Price'].sum()
print(f"\nTOTAL CAPITAL INVESTMENT VALUE: ${total_capital_investment:,.2f}")

# Top 3 Capital Asset Categories by Total Value
print("\n--- 5.2. Top 3 Capital Categories by Total Value ---")
top_cap_value = df_cap.groupby('Category')['Price'].sum().sort_values(ascending=False).head(3)
print(top_cap_value.to_string())

--- 5.1. Descriptive Statistics for Price (Capital) ---
count       30.00000
mean      2018.96000
std       4555.84825
min         30.00000
25%        201.50000
50%        443.75000
75%       1175.00000
max      23200.00000
Name: Price, dtype: float64

TOTAL CAPITAL INVESTMENT VALUE: $60,568.80

--- 5.2. Top 3 Capital Categories by Total Value ---
Category
Uncategorized    56908.8
1                 2460.0
12x5              1200.0


# 6️⃣ Final Insight: **Top Items Comparison**

***

### 📈 **Goal:** Provide immediate, high-value insights by comparing the most expensive individual items in both files. This comparison clearly distinguishes between:
* **High Opex:** The largest daily costs that impact cash flow.
* **High CapEx:** The most significant, long-term investments in factory capacity.

In [35]:
## 6. Top 5 Costliest Items Across Both Files

# 1. Top 5 Operating Expenses (Individual Transactions)
top_exp_items = df_exp.sort_values(by='Price / Amount', ascending=False).head(5)
print("--- 6.1. Top 5 Operating Expenses (Highest Individual Costs) ---")
print(top_exp_items[['Arabic Item', 'Category', 'Price / Amount']].to_string())

# 2. Top 5 Capital Investments
top_cap_items = df_cap.sort_values(by='Price', ascending=False).head(5)
print("\n--- 6.2. Top 5 Capital Investments (Highest Asset Value) ---")
print(top_cap_items[['Item Arabic', 'Category', 'Price']].to_string())

--- 6.1. Top 5 Operating Expenses (Highest Individual Costs) ---
   Arabic Item     Category  Price / Amount
24        شاحن    Equipment       1576000.0
25         رمل     Material        800000.0
34        اجار         Rent        500000.0
35      ابوعزت  Name/Person        375000.0
33        اجار         Rent        300000.0

--- 6.2. Top 5 Capital Investments (Highest Asset Value) ---
    Item Arabic       Category    Price
0             1  Uncategorized  23200.0
1             2  Uncategorized   9000.0
2             3  Uncategorized   6700.0
16           17  Uncategorized   6447.3
20           21  Uncategorized   2779.0
